In [ ]:
import pandas as pd
import numpy as np

In [ ]:
#nacist data
books = pd.read_csv('BX-Books.csv', encoding='iso-8859-1', sep=';', header=0,usecols=range(5))
ratings = pd.read_csv('BX-Book-Ratings.csv', encoding='iso-8859-1', sep=';', header=0)
mybooks = pd.read_csv('moje_vypujcky.txt', encoding='iso-8859-1',header=None,names=['ISBN'],dtype=str)


In [ ]:
#vybrání hodnocení pro mé knihy
ratings_my=pd.merge(ratings,mybooks,how="inner", on="ISBN")
#databáze knihy_vše - knihy_mé
new=pd.merge(ratings,ratings_my, how="inner",on="User-ID")

In [ ]:
#počet hodnocení u jednotlivé knihy
df2=ratings.groupby(["ISBN"]).count()
#počet_hodnocení u knih společně
df1=new.groupby(["ISBN_x","ISBN_y"]).count()
df3=pd.merge(df1.reset_index(),df2.reset_index(),left_on="ISBN_x",right_on="ISBN", how="left",suffixes=(1,2))
#kromě jiného tady mám ke kažné dvojici knih, které mě zajímají kolikrát se objevili společně a pak kolikrát se každá objevila zvlášt
df4=pd.merge(df3.reset_index(),df2.reset_index(),left_on="ISBN_y",right_on="ISBN", how="left",suffixes=(3,4))


In [129]:
# nový sloupeček "podobnost", který udává, jak jsou knihy od sebe vzdáleny v Jaccardově míře
df4["podobnost"]=df4["User-ID1"]/(df4["User-ID2"]+df4["User-ID"]-2*df4["User-ID1"]+1)


In [ ]:
result=pd.merge(df4,books,how="inner",left_on="ISBN_x",right_on="ISBN").sort_values(by="podobnost",ascending=False)
# ořezání o to, co nechci tisknout
result_f=result.loc[:,['ISBN', 'Book-Title',
       'Book-Author', 'Year-Of-Publication', 'Publisher',"podobnost"]]
# tak tohle už je tu jen proto, že nemáme vyčištěné data a k jedné knize můžeme mít víc ISBN, když se to stane, tak je prostě sečtu
# top kinhy jsou samozřejmě knihy z mého výběru, což je fajn, lze nastavit tak, aby se nezobrazili, ale schválně jsem to nechala 
result_f.groupby(["Book-Title","Book-Author"]).sum().sort_values(by="podobnost",ascending=False).head(20)
